In [1]:
import pandas as pd

data = pd.read_csv("/home/fm-pc-lt-228/Desktop/covid_nepali_tweets/covid19_tweeter_dataset (copy).csv")

In [2]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Sakonii/deberta-base-nepali", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Sakonii/deberta-base-nepali")

In [3]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
#     import pdb;pdb.set_trace()
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [4]:
data['word_embeddings'] = data['Tweet'].apply(get_bert_embedding_sentence)

In [5]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [6]:
X,y = data['word_embeddings'], data['Label']

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [8]:
svc = SVC(probability=True)

In [9]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [10]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [11]:
print(confusion_matrix(test_y, svc_pred))

[[2214   44  563]
 [ 336  428  535]
 [ 575   37 2348]]


In [12]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

          -1       0.71      0.78      0.74      2821
           0       0.84      0.33      0.47      1299
           1       0.68      0.79      0.73      2960

    accuracy                           0.70      7080
   macro avg       0.74      0.64      0.65      7080
weighted avg       0.72      0.70      0.69      7080



In [13]:
accuracy_score(test_y, svc_pred)

0.7048022598870056

In [14]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [16]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, random_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.5759887005649718
[[1390    0 1431]
 [ 214   32 1053]
 [ 304    0 2656]]
              precision    recall  f1-score   support

          -1       0.73      0.49      0.59      2821
           0       1.00      0.02      0.05      1299
           1       0.52      0.90      0.66      2960

    accuracy                           0.58      7080
   macro avg       0.75      0.47      0.43      7080
weighted avg       0.69      0.58      0.52      7080



# Naive Bayes

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [18]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.5983050847457627
[[1799  438  584]
 [ 302  597  400]
 [ 620  500 1840]]
              precision    recall  f1-score   support

          -1       0.66      0.64      0.65      2821
           0       0.39      0.46      0.42      1299
           1       0.65      0.62      0.64      2960

    accuracy                           0.60      7080
   macro avg       0.57      0.57      0.57      7080
weighted avg       0.61      0.60      0.60      7080



# Ada Boost

In [19]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [20]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.6367231638418079
[[1964  155  702]
 [ 367  443  489]
 [ 673  186 2101]]
              precision    recall  f1-score   support

          -1       0.65      0.70      0.67      2821
           0       0.57      0.34      0.43      1299
           1       0.64      0.71      0.67      2960

    accuracy                           0.64      7080
   macro avg       0.62      0.58      0.59      7080
weighted avg       0.63      0.64      0.63      7080



# Decision Tree

In [21]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [22]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.5362994350282486
[[1757   44 1020]
 [ 532  151  616]
 [1018   53 1889]]
              precision    recall  f1-score   support

          -1       0.53      0.62      0.57      2821
           0       0.61      0.12      0.20      1299
           1       0.54      0.64      0.58      2960

    accuracy                           0.54      7080
   macro avg       0.56      0.46      0.45      7080
weighted avg       0.55      0.54      0.51      7080



# Extra Trees Classifier

In [23]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [24]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.6692090395480226
[[1930   75  816]
 [ 282  395  622]
 [ 495   52 2413]]
              precision    recall  f1-score   support

          -1       0.71      0.68      0.70      2821
           0       0.76      0.30      0.43      1299
           1       0.63      0.82      0.71      2960

    accuracy                           0.67      7080
   macro avg       0.70      0.60      0.61      7080
weighted avg       0.68      0.67      0.65      7080



# Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000.0)

In [26]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.6990112994350283
[[2130  135  556]
 [ 308  528  463]
 [ 526  143 2291]]
              precision    recall  f1-score   support

          -1       0.72      0.76      0.74      2821
           0       0.66      0.41      0.50      1299
           1       0.69      0.77      0.73      2960

    accuracy                           0.70      7080
   macro avg       0.69      0.65      0.66      7080
weighted avg       0.70      0.70      0.69      7080



# KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [28]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.6413841807909605
[[2146  153  522]
 [ 454  454  391]
 [ 868  151 1941]]
              precision    recall  f1-score   support

          -1       0.62      0.76      0.68      2821
           0       0.60      0.35      0.44      1299
           1       0.68      0.66      0.67      2960

    accuracy                           0.64      7080
   macro avg       0.63      0.59      0.60      7080
weighted avg       0.64      0.64      0.63      7080



# MLP

In [29]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [30]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.6768361581920904
[[2012  218  591]
 [ 274  648  377]
 [ 536  292 2132]]
              precision    recall  f1-score   support

          -1       0.71      0.71      0.71      2821
           0       0.56      0.50      0.53      1299
           1       0.69      0.72      0.70      2960

    accuracy                           0.68      7080
   macro avg       0.65      0.64      0.65      7080
weighted avg       0.67      0.68      0.68      7080

